In [4]:
import os
import pytorch_lightning as pl
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from conditional_rate_matching import data_path

image_data_path = os.path.join(data_path,"raw")

class SimpleCNN(pl.LightningModule):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

    def training_step(self, batch, batch_idx):
        data, target = batch
        output = self(data)
        loss = nn.CrossEntropyLoss()(output, target)
        return {'loss': loss}

    def test_step(self, batch, batch_idx):
        data, target = batch
        output = self(data)
        loss = nn.CrossEntropyLoss()(output, target)
        acc = (output.argmax(dim=1) == target).float().mean()
        return {'test_loss': loss, 'test_acc': acc}

    def on_test_epoch_end(self):
        avg_loss = torch.stack(self.test_loss).mean()
        avg_acc = torch.stack(self.test_acc).mean()
        metrics = {'test_loss': avg_loss.item(), 'test_acc': avg_acc.item()}
        print(metrics)
        self.test_loss = []
        self.test_acc = []

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))])
        mnist_train = datasets.MNIST(image_data_path, train=True, download=True, transform=transform)
        return DataLoader(mnist_train, batch_size=64)

    def test_dataloader(self):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))])
        mnist_test = datasets.MNIST(image_data_path, train=False, download=True, transform=transform)
        return DataLoader(mnist_test, batch_size=64)

In [5]:

# Create the model and trainer
model = SimpleCNN()
trainer = pl.Trainer(max_epochs=3)

# Train and test the model
trainer.fit(model)
trainer.test(model)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layer1 | Sequential | 832   
1 | layer2 | Sequential | 51.3 K
2 | fc1    | Linear     | 3.1 M 
3 | fc2    | Linear     | 10.0 K
--------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.796    Total estimated model params size (MB)


Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=3` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\cesar\anaconda3\envs\rate_matching\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |                                                                                       | 0/? [00:00<…

AttributeError: 'SimpleCNN' object has no attribute 'test_loss'